In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import re
from IPython.display import display, HTML
import HTMLParser
import itertools
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import getpass

# Start an SQL database

In [2]:
# Create a Postgresql
dbname = 'art_kickstarter'
username = getpass.getpass('username:')
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
if not database_exists(engine.url):
    create_database(engine.url)
print 'Database exist? '+ str(database_exists(engine.url))


username:········
Database exist? True


In [3]:
import time
print (time.strftime("%Y%m%d"))

20170909


In [4]:
con = None
con = psycopg2.connect(database = dbname, user = username)

cursor = con.cursor()
table_name = 'kickstarter_'+str(time.strftime("%Y%m%d"))
command = "DROP TABLE IF EXISTS "+table_name+";"
cursor.execute(command)
con.commit()
cursor.close()


# Dictionary for category ID to scrap

In [5]:
category_dict = {'Art': '1', 'Comics': '3', 'Crafts': '26', 'Dance': '6', 'Design': '7', 'Fashion': '9',
                 'Film_and_Video':'11', 'Food': '10', 'Games':'12', 'Journalism':'13', 'Music':'14',
                'Photography':'15', 'Publishing':'18','Technology':'16', 'Theater':'17'}

# Scraper protytpe(s) (to be updated)

In [6]:
def scraper_prototype(text_profile,prefix):
    
    if '"name"' in text_profile:
        text_profile = re.sub('"','',text_profile)
    
    text2 = text_profile.split(',')
    dummy2 = [re.findall('[A-Za-z0-9_]+:', ww) for ww in text2]
    itemname = [dum_part[0] for dum_part in dummy2 if len(dum_part)>0]
    ans_array = [re.search(xxy1+'(.*?)'+xxy2,text_profile).group(1).rstrip(',') 
                for xxy1,xxy2 in zip(itemname[:-1],itemname[1:])]    
    itemnames = [x.rstrip(':') for x in itemname]

    info_dict = dict()
    for item,ans in zip(itemnames,ans_array):
        info_dict[prefix+'_'+item] = ans
    return info_dict


# Loading the project info on a page using a scraper prototype and save into database.

In [7]:
time_stamp = time.strftime("%d/%m/%Y %H:%M:%S")

for category_key in category_dict:

    # === Create the table (if not exist) ===
    category = category_key
    #table_name = category.lower()+"_table"
    print(category_key) 

    # === Start scraping ===
    total_num_of_items = 0
    for pageno in range(100):

        r = urllib.urlopen('https://www.kickstarter.com/discover/advanced?state=live&category_id='\
        +category_dict[category]+'&sort=popularity&seed=2506137&page='+str(pageno)).read()
        soup = BeautifulSoup(r, 'html.parser')

        kk = soup.find_all('div', class_="js-react-proj-card")
        total_num_of_items += len(kk)
        
        # Last page to scrap
        if len(kk)==0:
            print('page number = %s' % str(pageno)),
            print(', total number of items = %s' % str(total_num_of_items))
            break

        for i in range(len(kk)):
            whole_string = str(kk[i]).replace('&amp','').replace('&quot;','')

            # === Overall project info ===
            if '"name"' in whole_string:
                project_info = '"name"'+ re.findall('"name"(.+?)"creator"',whole_string)[0]
            else:
                project_info = 'name'+ re.findall('name(.+?)creator',whole_string)[0]
            project_info_dict = scraper_prototype(project_info,'project')
            df1 = pd.DataFrame.from_dict(project_info_dict,orient='index').transpose()

            # === Scrap creater string ===
            if '"creator"' in whole_string:
                creator_string = re.findall(r'\"creator\"\:\{(.+?)\}',whole_string)[0]
            else:
                creator_string = re.findall(r'creator\:\{(.+?)\}',whole_string)[0]
            creator_info_dict = scraper_prototype(creator_string,'creator')
            creator_info_dict['creator_slug'] = creator_info_dict.get('creator_slug', 'Null')            
            df2 = pd.DataFrame.from_dict(creator_info_dict,orient='index').transpose()

            # === Scrap location string ===
            if '"location"' in whole_string:
                location_string = re.findall(r'\"location\"\:\{(.+?)\}',whole_string)[0]
            else:
                location_string = re.findall(r'location\:\{(.+?)\}',whole_string)[0]
            #location_string = re.findall(r'\"location\"\:\{(.+?)\}',whole_string)
            location_info_dict = scraper_prototype(location_string,'location')
            df3 = pd.DataFrame.from_dict(location_info_dict,orient='index').transpose()             

            # === Scrap profile string ===
            if '"profile"' in whole_string:
                profile_string = re.findall(r'\"profile\"\:\{(.+?)\}',whole_string)[0]
            else:
                profile_string = re.findall(r'profile\:\{(.+?)\}',whole_string)[0]

            profile_info_dict = scraper_prototype(profile_string,'profile')
            df4 = pd.DataFrame.from_dict(profile_info_dict,orient='index').transpose()

            df = pd.concat((df1,df2,df3,df4),axis=1)
            df['category'] = category_key
            df['retrieval_time'] = time_stamp

            # Record down columns
            if pageno == 0 and i==0:
                df_columns = df.columns

            #print 'Finish concatenating to df'
            try:            
                df.to_sql(table_name, engine, if_exists='append')
                #print 'Finish putting into art_table'            
            except:
                to_eliminate = set(df.columns.values) - set(df_columns.values)
                for to_drop in to_eliminate:
                    df.drop(to_drop,axis=1,inplace=True)
                df.to_sql(table_name, engine, if_exists='append')
                #print 'Successfully handle exception'


Theater
page number = 7 , total number of items = 75
Publishing
page number = 37 , total number of items = 442
Comics
page number = 13 , total number of items = 150
Journalism
page number = 6 , total number of items = 67
Music
page number = 35 , total number of items = 412
Crafts
page number = 11 , total number of items = 127
Fashion
page number = 31 , total number of items = 368
Art
page number = 25 , total number of items = 298
Dance
page number = 3 , total number of items = 35
Photography
page number = 8 , total number of items = 96
Food
page number = 24 , total number of items = 281
Design


KeyboardInterrupt: 

# Check what's on the SQL database

In [8]:
pd.options.display.max_columns = 100

sql_query = """
SELECT * FROM %s;
"""
birth_data_from_sql = pd.read_sql_query(sql_query % (table_name),con)

print(birth_data_from_sql.shape)

birth_data_from_sql

(2615, 55)


,index,project_pledged,project_name,project_currency_symbol,project_slug,project_currency_trailing_code,project_disable_communication,project_country,project_state,project_blurb,project_state_changed_at,project_currency,project_launched_at,project_static_usd_rate,project_goal,project_backers_count,project_is_starrable,project_deadline,project_created_at,project_staff_pick,creator_name,creator_small,creator_id,creator_avatar,creator_slug,creator_is_registered,location_localized_name,location_state,location_name,location_country,location_urls,location_id,location_type,location_short_name,location_slug,location_displayable_name,location_is_root,profile_blurb,profile_link_text,profile_link_background_color,profile_background_image_opacity,profile_id,profile_project_id,profile_show_feature_image,profile_should_show_feature_image_section,profile_text_color,profile_name,profile_link_text_color,profile_background_color,profile_state_changed_at,profile_link_url,profile_state,profile_feature_image_attributes,category,retrieval_time
0,0,11633.0,The Television Workshop: The New Home,£,the-television-workshop-the-new-home,false,false,GB,live,The BAFTA award-winning Television Workshop ha...,1503058655,GBP,1503058654,1.2905274,15000.0,170,true,1505650654,1484837880,false,Nic Harvey,https://ksr-ugc.imgix.net/assets/017/196/425/f...,1904961866,{thumb:https://ksr-ugc.imgix.net/assets/017/19...,Null,true,Nottingham,England,Nottingham,GB,{web:{discover:https://www.kickstarter.com/dis...,30720,Town,"Nottingham, UK",nottingham-gb,"Nottingham, UK",false,null,null,null,0.8,2842967,2842967,false,true,null,null,null,null,1484837880,null,inactive,{image_urls:{default:https://ksr-ugc.imgix.net...,Theater,09/09/2017 16:49:16
1,0,16391.0,The Conjuror's Club,$,the-conjurors-club,true,false,US,live,Hidden in New York City lies a century-old sec...,1503509000,USD,1503509000,1.0,18000.0,141,true,1506101000,1499374311,false,Vinny DePonto,https://ksr-ugc.imgix.net/assets/006/142/315/7...,138230327,{thumb:https://ksr-ugc.imgix.net/assets/006/14...,vinnydeponto,true,New York,NY,New York,US,{web:{discover:https://www.kickstarter.com/dis...,2459115,Town,"New York, NY",new-york-ny,"New York, NY",false,null,null,null,0.8,3064389,3064389,false,true,null,null,null,null,1499374311,null,inactive,{image_urls:{default:https://ksr-ugc.imgix.net...,Theater,09/09/2017 16:49:16
2,0,1221.0,Queen Cunt - Sacred or Profane? Comedy Sketch ...,£,queen-cunt-sacred-or-profane-comedy-sketch-show,false,false,GB,live,"Creating a new, absurd, ecofeminist comedy. A ...",1503042686,GBP,1503042685,1.2905274,5000.0,46,true,1505683800,1502783597,false,Deborah Antoinette Ward,https://ksr-ugc.imgix.net/assets/017/929/047/8...,2027984945,{thumb:https://ksr-ugc.imgix.net/assets/017/92...,Null,true,Bristol,England,Bristol,GB,{web:{discover:https://www.kickstarter.com/dis...,13963,Town,"Bristol, UK",bristol-city-of-bristol-england,"Bristol, UK",false,null,null,null,0.8,3108925,3108925,false,true,null,null,null,null,1502783597,null,inactive,{image_urls:{default:https://ksr-ugc.imgix.net...,Theater,09/09/2017 16:49:16
3,0,1526.0,The New York Sorta Marathon,$,the-new-york-sorta-marathon,true,false,US,live,The first ever SHORT marathon that anyone can ...,1503926081,USD,1503926081,1.0,8000.0,37,true,1506124800,1501882784,true,Zach Broussard,https://ksr-ugc.imgix.net/assets/006/039/496/0...,1683069409,{thumb:https://ksr-ugc.imgix.net/assets/006/03...,Null,true,New York,NY,New York,US,{web:{discover:https://www.kickstarter.com/dis...,2459115,Town,"New York, NY",new-york-ny,"New York, NY",false,null,null,null,0.8,3097665,3097665,false,true,null,null,null,null,1501882784,null,inactive,{image_urls:{default:https://ksr-ugc.imgix.net...,Theater,09/09/2017 16:49:16
4,0,2881.0,The Teenage Textbook Musical Showcase,$,the-teenage-textbook-musical-showcase,true,false,SG,live,"Follow Mui Ee as she defeats the bad guy, fall...",1504666038,SGD,1504666037,0.7366298,4500.0,25,true,1506268800,1504506991,false,Go! Theatre